# Enriching data from BGB with ship IDs from DAS

Version 0.1

*Gerhard de Kok*

This script is an experiment to enrich the data from the Bookkeeper-General Batavia dataset (BGB) with links to the data in the Dutch-Asiatic Shipping dataset (DAS).


### The problem

BGB has as list of voyages with associated internal ship IDs. Some of the ships in BGB are also present in DAS, which only contains intercontinental voyages. If a voyage (between Europe and Asia) is present in both BGB and DAS, a connection is (usually) already made between the BGB ship ID and the DAS ship ID. For about 14,500 voyages in BGB, such a connection is not present. Most of these voyages were intra-Asiatic and are (thus) not present in DAS. However, many of the ships used for these intra-Asiatic voyages are present in DAS. The challenge is to link BGB ship IDs to DAS ship IDs in such cases.


### Proposed solution

The present script tries to resolve this issue in a rule-based manner. Basically, the rules are as follows:

> For each voyage in BGB, look up the name of the ship involved
> See if a ship with a similar name is present in DAS
> See if the DAS ship was active in the same period as the BGB voyage (currently defined as first mention in DAS + 20 years)
> If so, provisionally link the BGB ship ID to the DAS ship ID


### Results

For now, the script matched 7264 BGB voyages to 752 unique DAS ships. Some of the results are problematic:

* The rules mark 'Anna Catharina' and 'Susanna Catharina' as the same ship. This can be resolved by decreasing the sensitivity for fuzzy matching (for which difflib in the Python standard library is used)
* For some very common ship names (Hoop), the results are not always correct

Some rule-based solutions are possible. I will add a refinement of the rules in a further iteration of this script.

In [1]:
# Import necessary modules
import pandas as pd
import difflib

In [2]:
# First, load the entire BGB database and the DAS database (Excel format)
bgb = pd.ExcelFile('bgb.xlsx')
das = pd.ExcelFile('das.xlsx')

# Parse the Excel sheets we will be using from BGB
# For the bgb_relations dataframe, some operations are needed to convert IDs to integers
bgb_ships = bgb.parse('bgb_ship')
bgb_ships = bgb_ships.set_index('id')
bgb_relations = bgb.parse('bgb_relVoyageShip')
bgb_relations.dropna(subset=['voyId'], how='all', inplace=True)
bgb_relations['voyId'] = pd.to_numeric(bgb_relations['voyId'], downcast='integer', errors='coerce')
bgb_relations = bgb_relations.set_index('id')
bgb_voyages = bgb.parse('bgb_voyage')
bgb_voyages = bgb_voyages.set_index('voyId')

# From DAS, we need the list of ship names ...
# ... and the years between which these ships were employed by the VOC
das_ships = das.parse('shipNameVariant')
das_ships = das_ships.set_index('shipNameVariantID')
das_voyages = das.parse('das_voyage')
das_voyages = das_voyages.set_index('voyId')

In [3]:
# Now I want a list of individual ships and the years between which they were employed
# Converting the date to datetime is not possible, since many voyages took place before 1677 (out of bounds)
# This means we lose vectorization advantages anyway, so I'll generate a dataframe using a Python loop 

# Create an empty list to hold the data on ships and dates
das_ship_dates = []

# Populate the list with data from DAS
for voyage in das_voyages.index:
    current_ship_id = das_voyages.loc[voyage, 'shipID']
    current_ship_name_id = das_voyages.loc[voyage, 'shipName']
    current_ship_departure = das_voyages.loc[voyage, 'voyDepartureEDTF']
    current_ship_arrival = das_voyages.loc[voyage, 'voyArrivalDateEDTF']
    current_ship_name = das_ships.loc[current_ship_name_id, 'shipNameVariant']
    
    # Convert dates to integer (datetime not possible w/o workarounds)
    current_ship_departure = str(current_ship_departure)
    current_ship_departure = current_ship_departure[:4]
    current_ship_arrival = str(current_ship_arrival)
    current_ship_arrival = current_ship_arrival[:4]

    # Construct a list with data on this voyage
    this_voyage = (voyage, current_ship_name, current_ship_id, current_ship_name_id, current_ship_departure, current_ship_arrival)
    
    # Append that list to the aforementioned list (of lists)
    das_ship_dates.append(this_voyage)
    
# Create a Pandas dataframe from the list of lists
fulldata = pd.DataFrame.from_records(das_ship_dates, columns=['DasID', 'Shipname', 'DasShipID', 'DasShipNameVariant', 'Startyear', 'Endyear'])
fulldata = fulldata.set_index('DasID')

# Convert to yearcolumns to numeric values (they include some messed up data)
fulldata['Startyear'] = pd.to_numeric(fulldata['Startyear'], errors='coerce')
fulldata['Endyear'] = pd.to_numeric(fulldata['Endyear'], errors='coerce')

# Now we can drop NaNs (error coercion made these NaN)
# And subsequently convert from float to int (due to messed up data, to_numeric couldn't do this)
fulldata.dropna(how='any', inplace=True)
fulldata['Startyear'] = fulldata['Startyear'].astype(int)
fulldata['Endyear'] = fulldata['Endyear'].astype(int)


In [4]:
# We now have all the data to make a dataframe with: DAS IDs, shipnames, ... 
# ... first year the ship was active, last year the ship was active (on intercontinental voyages)
# Caveat: NaN-values are excluded
# Caveat (2): renaming of ships not taken into account (although all rows are individual ships)
summary = fulldata.groupby(['DasShipID']).agg({'Shipname':'first', 'Startyear':'min', 'Endyear':'max'})
summary

,Shipname,Startyear,Endyear
DasShipID,,,
DAS_ship0001,'s Heer Arendskerke,1725,1742
DAS_ship0002,'s Lands Welvaren,1763,1773
DAS_ship0003,'s-Graveland,1659,1660
DAS_ship0004,'s-Graveland,1723,1726
DAS_ship0005,'s-Gravenhage,1628,1635
...,...,...,...
DAS_ship1851,President,1675,1675
DAS_ship1852,Schaapherder,1692,1693
DAS_ship1854,Toevalligheid,1745,1745


In [5]:
# Generate a list with unique ship names in DAS for fuzzy matching later on
dasshiplist = list(summary['Shipname'].unique())

In [6]:
# Now we can turn our attention to the BGB data
# Let's check which voyages are missing a link to DAS
missing_data = bgb_relations[bgb_relations['DAS_shipID'].isna()]

# Add the BGB voyage IDs for which a link is missing to a list
missinglist = list(missing_data['voyId'].unique())

In [7]:
# Loop over the list with BGB voyages IDs for which links are missing
# And gather the necessary information to apply the rules

# Define an empty list to hold the results
voyagedata = []

for voyage in missinglist:

    # In some instances, the BGB voyage ID is empty
    if pd.isnull(voyage):
        continue
        
    # Get the BGB booking year of this voyage (with try/except since some voyages do not exist in BGB voyages table)
    checkvoyage = int(voyage)
    try:
        bookingyear = bgb_voyages.loc[checkvoyage, 'voyBookingYear']
    except:
        continue
    
    # Get the BGB shipname for this voyage (or the first if more than one ship was involved) (with try/except for data errors)
    try:
        shipid = int(bgb_relations.loc[bgb_relations['voyId'] == voyage]['shipId'])
        shipname = bgb_ships.loc[shipid, 'naam']
    except:
        continue
    
    # Check for a closely matching shipname in DAS
    checking = difflib.get_close_matches(shipname, dasshiplist, n=3, cutoff=0.8)
    if not checking:
        continue
    
    # There may be multiple ships that fuzzy match with the BGB name, I use only the closest match
    # The name of the closest match may be given to multiple ships. Here, I pull the year of first mention
    # in DAS for each of these ships from the dataframe
    dasyears = list(summary.loc[summary['Shipname'] == checking[0]]['Startyear'])
    
    # For each of these years of first mention, check to see if it falls within the range of 20 years
    # (difference between BGB booking year and DAS year of first mention)
    if dasyears:
        for year in dasyears:
            delta = bookingyear - year
            
            # If it falls within the range:
            if (delta > -1) & (delta < 20):
                # Lookup corresponding DAS shipID
                dasshipid = summary.loc[(summary['Shipname'] == checking[0]) & (summary['Startyear'] == year)].index[0]                                      

                # Construct a list with relevant data on this voyage
                this_voyage = (checkvoyage, shipid, dasshipid, shipname, checking[0], bookingyear, year)
                
                # Add that list to the results list
                voyagedata.append(this_voyage)
                continue

# Build a new dataframe from the results
matched = pd.DataFrame.from_records(voyagedata, columns=['BGB Voyage ID', 'BGB ship ID', 'DAS Ship ID', 'BGB ship name', 'DAS ship name', 'BGB Booking year', 'DAS first seen'])


In [8]:
matched

,BGB Voyage ID,BGB ship ID,DAS Ship ID,BGB ship name,DAS ship name,BGB Booking year,DAS first seen
0,99405,3112,DAS_ship0599,Hindeloopen,Hindeloopen,1790,1778
1,99407,3115,DAS_ship0892,Leviathan,Leviathan,1790,1787
2,99408,3116,DAS_ship1575,Vredenburg,Vredenburg,1790,1785
3,99409,3117,DAS_ship1312,Schelde,Schelde,1790,1784
4,99411,3121,DAS_ship0656,Horssen,Horssen,1790,1785
...,...,...,...,...,...,...,...
7259,118245,3554,DAS_ship0966,Meermin,Meermin,1763,1761
7260,118250,3077,DAS_ship1805,Zuiderburg,Zuiderburg,1763,1744
7261,118243,3738,DAS_ship1073,Noord Nieuwland,Noord Nieuwland,1763,1750
7262,118252,3862,DAS_ship1366,Sloterdijk,Sloterdijk,1763,1748


In [9]:
matched.to_excel("matched.xlsx")